<div class="alert alert-block alert-success">
<h1> DATA MINING - PROJECT </h1>
<h2> Group 19 </h2>
<h3> 20225/2026 </h3>

Student Name - António Parafita <br>
Student id - 20250363 <br>

Student Name - Diogo Montenegro <br>
Student id - 20250491 <br>

Student Name - Francisco <br>
Student id - 20250482 <br>

## **Index**<br>

[1. **Imports**](#1st-bullet)<br>
- [1.1 Import libraries](#2nd-bullet)<br>
- [1.2 Import Data](#3rd-bullet)<br>
    
[2. **Exploratory Analysis**](#4th-bullet)<br>
- [2.1 Minimal Preprocessing](#5th-bullet)<br>
    - [2.1.1 Data Types](#6th-bullet)<br>
    - [2.1.2 Duplicates, Zeros, Nan and Cegative values](#7th-bullet)<br>
    - [2.1.3 New features](#8th-bullet)<br>
    - [2.1.4 New Flights data set aggregated by customer](#9th-bullet)<br>

- [2.2 Univariate Analysis](#10th-bullet)<br>
    - [2.2.1 In-Depth Numerical Analysis](#11th-bullet)<br>
    - [2.2.1.1 Visualization](#12th-bullet)<br>
    - [2.2.2 In-Depth Categorical Analysis](#13th-bullet)<br>
    - [2.2.1.1 Visualization](#14th-bullet)<br>
- [2.3 Bivariate Analysis](#15th-bullet)<br>
    - [2.3.1 Numeric-Numeric](#16th-bullet)<br>
    - [2.3.2 Categorical-Categorical](#17th-bullet)<br>
    - [2.3.3 Numeric-Categorical](#18th-bullet)<br>
        - [2.3.3.1 Seasonality check](#19th-bullet)<br>
- [2.4 Outlier Detection](#20th-bullet)<br>
    

<a class="anchor" id="1st-bullet">    </a>
# 1. Imports

<a class="anchor" id="2st-bullet">    </a>
## 1.1 Import libraries

In [13]:
import sqlite3
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA

from itertools import product

#maps
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from IPython.display import IFrame

from ydata_profiling import ProfileReport

# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Setting seaborn style
sns.set_style("ticks")

<a class="anchor" id="3st-bullet">    </a>
## 1.2 Import Data

In [14]:
customers = pd.read_csv('data/DM_AIAI_CustomerDB.csv', index_col = 0)
flights = pd.read_csv('data/DM_AIAI_FlightsDB.csv')

In [15]:
df_c = customers.copy()

In [16]:
df_f = flights.copy()

<a class="anchor" id="5st-bullet">    </a>
## 2.1 Preprocessing

<a class="anchor" id="6st-bullet">    </a>
### 2.1.1 Data types

**Set 'EnrollmentDateOpening', 'CancellationDate' & 'YearMonthDate' type to *datetime***

In [17]:
df_c['EnrollmentDateOpening'] = pd.to_datetime(
    df_c['EnrollmentDateOpening'],
    errors='coerce'
)

df_c['CancellationDate'] = pd.to_datetime(
    df_c['CancellationDate'],
    errors='coerce'
)

df_f['YearMonthDate'] = pd.to_datetime(
    df_f['YearMonthDate'],
    errors='coerce'
)

**Changing data type of numFlights & numFlightsWithCompanions to int**

In [18]:
df_f['NumFlights'] = df_f['NumFlights'].astype(int)
df_f['NumFlightsWithCompanions'] = df_f['NumFlightsWithCompanions'].astype(int)

<a class="anchor" id="7st-bullet">    </a>
### 2.1.2 Duplicates, Zeros, Nan & negative values

**Checking Nan values**

In [19]:
# replace "" by nans
df_c.replace("", np.nan, inplace=True)
# count of missing values
df_c.isna().sum()

Loyalty#                       0
First Name                     0
Last Name                      0
Customer Name                  0
Country                        0
Province or State              0
City                           0
Latitude                       0
Longitude                      0
Postal code                    0
Gender                         0
Education                      0
Location Code                  0
Income                        20
Marital Status                 0
LoyaltyStatus                  0
EnrollmentDateOpening          0
CancellationDate           14613
Customer Lifetime Value       20
EnrollmentType                 0
dtype: int64

In [20]:
# replace "" by nans
df_f.replace("", np.nan, inplace=True)

# count of missing values
df_f.isna().sum()

Loyalty#                    0
Year                        0
Month                       0
YearMonthDate               0
NumFlights                  0
NumFlightsWithCompanions    0
DistanceKM                  0
PointsAccumulated           0
PointsRedeemed              0
DollarCostPointsRedeemed    0
dtype: int64

**Checking for duplicates and dropping them**

In [21]:
df_c.duplicated().sum()

np.int64(0)

In [22]:
df_f.duplicated().sum()

np.int64(2903)

In [27]:
df_f[df_f.duplicated(keep=False)]


,Loyalty#,Year,Month,YearMonthDate,NumFlights,NumFlightsWithCompanions,DistanceKM,PointsAccumulated,PointsRedeemed,DollarCostPointsRedeemed
450,727091,2021,12,2021-12-01,0,0,0.0,0.0,0.0,0.0
535,369638,2021,12,2021-12-01,0,0,0.0,0.0,0.0,0.0
762,750578,2020,6,2020-06-01,0,0,0.0,0.0,0.0,0.0
941,547522,2020,6,2020-06-01,0,0,0.0,0.0,0.0,0.0
952,819842,2020,6,2020-06-01,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
607696,960050,2019,12,2019-12-01,0,0,0.0,0.0,0.0,0.0
608110,981508,2019,12,2019-12-01,0,0,0.0,0.0,0.0,0.0
608111,981508,2019,12,2019-12-01,0,0,0.0,0.0,0.0,0.0
608263,990512,2019,12,2019-12-01,0,0,0.0,0.0,0.0,0.0


In [28]:
df_f = df_f.drop_duplicates()
df_f.duplicated().sum()

np.int64(0)

— We should drop them because a customer shouldn't have the same month of the same year registered more than once

**Checking if Loyalty# has repeated values**

In [33]:
# How many Loyalty# have duplicates?
print(df_c['Loyalty#'].value_counts()[df_c['Loyalty#'].value_counts() > 1].size)
print(df_f['Loyalty#'].value_counts()[df_f['Loyalty#'].value_counts() > 1].size)

0
16574


In [30]:
#checking if there are extra ids in flights dataset
extra_ids = set(df_f['Loyalty#']) - set(df_c['Loyalty#'])
print( len(extra_ids))

0


In [31]:
#checking how many customer ids don't have flights
ids_without_flights = set(df_c['Loyalty#']) - set(df_f['Loyalty#'])
print( len(ids_without_flights))

20


In [32]:
# 1) Find problematic Loyalty# values in the df_c (appear more than once)
dupe_ids_cust = (
    df_c['Loyalty#']
    .value_counts()
    .loc[lambda s: s > 1]
    .index
)

# 2) Completely remove those values from both datasets
df_c = df_c[~df_c['Loyalty#'].isin(dupe_ids_cust)].copy()
df_f = df_f[~df_f['Loyalty#'].isin(dupe_ids_cust)].copy()

**Checking zeros and negative values**

Customers data set

In [34]:
# Check for zeros in customer dataset
(df_c == 0).sum()

Loyalty#                      0
First Name                    0
Last Name                     0
Customer Name                 0
Country                       0
Province or State             0
City                          0
Latitude                      0
Longitude                     0
Postal code                   0
Gender                        0
Education                     0
Location Code                 0
Income                     4198
Marital Status                0
LoyaltyStatus                 0
EnrollmentDateOpening         0
CancellationDate              0
Customer Lifetime Value       0
EnrollmentType                0
dtype: int64

In [35]:
# Check for negative values in customer dataset
(df_c.select_dtypes(include=['number']) < 0).sum()

Loyalty#                       0
Latitude                       0
Longitude                  16594
Income                         0
Customer Lifetime Value        0
dtype: int64

Flight data set

In [36]:
# Check for zeros in flights dataset
(df_f == 0).sum()

Loyalty#                         0
Year                             0
Month                            0
YearMonthDate                    0
NumFlights                  303071
NumFlightsWithCompanions    447294
DistanceKM                  291496
PointsAccumulated           291496
PointsRedeemed              561780
DollarCostPointsRedeemed    561780
dtype: int64

In [37]:
# Check for negative values in flights dataset
(df_f.select_dtypes(include=['number']) < 0).sum()

Loyalty#                    0
Year                        0
Month                       0
NumFlights                  0
NumFlightsWithCompanions    0
DistanceKM                  0
PointsAccumulated           0
PointsRedeemed              0
DollarCostPointsRedeemed    0
dtype: int64